In [1]:
#In this cell I will import all the libraries I will need for this project

import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#In this cell I will load the New York City neighborhood data

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
#In this cell I will extract just the necessary neighborhood data from the New York City dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [4]:
#In this cell I will create a DataFrame to store the neighborhood data in

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
#In this cell I will fill the DataFrame with the neighborhoods data

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
#In this cell I will define my Foursquare credentials

CLIENT_ID = 'OD52D2OKTNM0M0H4NJGTXCKQSJXK02ZMY1YRCHJ23NF3V0GT' 
CLIENT_SECRET = 'DOC2PICIMX4IPFVYXICBYQGHYADDIPOI2PA1UY45422O3TUZ' 
VERSION = '20180605' 

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: OD52D2OKTNM0M0H4NJGTXCKQSJXK02ZMY1YRCHJ23NF3V0GT
CLIENT_SECRET:DOC2PICIMX4IPFVYXICBYQGHYADDIPOI2PA1UY45422O3TUZ


In [88]:
#In this cell I will get the latitude and longitude of our first neighborhood, Wakefield

neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] 
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] 

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Tribeca are 40.721521967443216, -74.01068328559087.


In [89]:
#In this cell I will search for the term "Hospital" around a 1 mile radius of our first neighborhood of Wakefield

search_query='Hospital'
LIMIT = 100 
radius = 1610

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude,
    search_query,
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/search?&client_id=OD52D2OKTNM0M0H4NJGTXCKQSJXK02ZMY1YRCHJ23NF3V0GT&client_secret=DOC2PICIMX4IPFVYXICBYQGHYADDIPOI2PA1UY45422O3TUZ&v=20180605&ll=40.721521967443216,-74.01068328559087&query=Hospital&radius=1610&limit=100'

In [90]:
#In this cell we can see the results for hospitals near Wakefield

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d64485086bc49002c3c102e'},
 'response': {'venues': [{'id': '4a82ef0af964a52092f91fe3',
    'name': 'NewYork-Presbyterian-Lower Manhattan Hospital',
    'location': {'address': '170 William St',
     'crossStreet': 'at Gold St',
     'lat': 40.71066770948401,
     'lng': -74.0054107430059,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71066770948401,
       'lng': -74.0054107430059}],
     'distance': 1287,
     'postalCode': '10038',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['170 William St (at Gold St)',
      'New York, NY 10038',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15

In [91]:
# In this cell I transform the results into a DataFrame

venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a82ef0af964a52092f91fe3,170 William St,US,New York,United States,at Gold St,1287,"[170 William St (at Gold St), New York, NY 100...","[{'label': 'display', 'lat': 40.71066770948401...",40.710668,-74.005411,10038,NY,NewYork-Presbyterian-Lower Manhattan Hospital,v-1566853200,NaN
1,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bad1a23f964a520b62d3be3,NaN,US,New York,United States,NaN,1379,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.70998454215684...",40.709985,-74.004720,NaN,NY,NY Presbyterian Hospital Lower Manhattan Hosp ER,v-1566853200,NaN
2,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b1c0e79f964a5205c0124e3,5 Lispenard St.,US,New York,United States,Near Corner of 6th Avenue and Lispenard,501,[5 Lispenard St. (Near Corner of 6th Avenue an...,"[{'label': 'display', 'lat': 40.72057803284648...",40.720578,-74.004869,10013,NY,Tribeca Soho Animal Hospital,v-1566853200,33365451
3,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bc3820374a9a593eba5d4f6,"21 South End Ave., Ground Floor",US,New York,United States,W Thames St,1607,"[21 South End Ave., Ground Floor (W Thames St)...","[{'label': 'display', 'lat': 40.70816281460506...",40.708163,-74.017927,10280,NY,Battery Park Veterinary Hospital,v-1566853200,33365449
4,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae07a92f964a520857f21e3,504 Hudson St,US,New York,United States,Christopher St,1354,"[504 Hudson St (Christopher St), New York, NY ...","[{'label': 'display', 'lat': 40.73324665779437...",40.733247,-74.006381,10014,NY,Greenwich Village Animal Hospital,v-1566853200,NaN


In [92]:
#In this cell I define a way for us to view the category of each venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        print ('error')
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [93]:
#In this cell I filter and clean the columns so we can see our results clearly

venues = results['response']['venues']
nearby_hospitals = json_normalize(venues)

filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals =nearby_hospitals.loc[:, filtered_columns]
nearby_hospitals['categories'] = nearby_hospitals.apply(get_category_type, axis=1)
nearby_hospitals.columns = [col.split(".")[-1] for col in nearby_hospitals.columns]

nearby_hospitals

,name,categories,lat,lng
0,NewYork-Presbyterian-Lower Manhattan Hospital,Hospital,40.710668,-74.005411
1,NY Presbyterian Hospital Lower Manhattan Hosp ER,Emergency Room,40.709985,-74.004720
2,Tribeca Soho Animal Hospital,Veterinarian,40.720578,-74.004869
3,Battery Park Veterinary Hospital,Veterinarian,40.708163,-74.017927
4,Greenwich Village Animal Hospital,Veterinarian,40.733247,-74.006381
5,Fountain Pen Hospital,Miscellaneous Shop,40.713934,-74.007508
6,Reade Street Animal Hospital,Veterinarian,40.716770,-74.010240
7,Seaport Animal Hospital,Veterinarian,40.708821,-74.003925
8,Beekman-Downtown Hospital Staff Residence,Residential Building (Apartment / Condo),40.709372,-74.004819
9,The Hospital,Brewery,40.711840,-74.010959


In [86]:
#In this cell I will create a function to run the rest of the neighborhoods through the same mile-wide hospital search

def getNearbyHospitals(names, latitudes, longitudes, radius=1610, LIMIT=100):
    
    hospitals_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
       
        url ='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude,
            search_query,
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['venues']
        
        hospitals_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_hospitals = pd.DataFrame([item for hospitals_list in hospitals_list for item in hospitals_list])
    nearby_hospitals.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_hospitals)

In [87]:
#In this cell I will put our results into the nearby_hospitals DataFrame

nearby_hospitals = getNearbyHospitals(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
nearby_hospitals

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Montefiore Wakefield Emergency Room (NYC EMS H...,40.893680,-73.860706,Emergency Room
1,Wakefield,40.894705,-73.847201,Montefiore Medical Center - Wakefield Campus,40.894207,-73.860790,Hospital
2,Co-op City,40.874294,-73.829939,Montefiore Wakefield Emergency Room (NYC EMS H...,40.893680,-73.860706,Emergency Room
3,Co-op City,40.874294,-73.829939,Montefiore Medical Center - Wakefield Campus,40.894207,-73.860790,Hospital
4,Eastchester,40.887556,-73.827806,Montefiore Wakefield Emergency Room (NYC EMS H...,40.893680,-73.860706,Emergency Room
5,Eastchester,40.887556,-73.827806,Montefiore Medical Center - Wakefield Campus,40.894207,-73.860790,Hospital
6,Fieldston,40.895437,-73.905643,Montefiore Wakefield Emergency Room (NYC EMS H...,40.893680,-73.860706,Emergency Room
7,Fieldston,40.895437,-73.905643,Montefiore Medical Center - Wakefield Campus,40.894207,-73.860790,Hospital
8,Riverdale,40.890834,-73.912585,Montefiore Wakefield Emergency Room (NYC EMS H...,40.893680,-73.860706,Emergency Room
9,Riverdale,40.890834,-73.912585,Montefiore Medical Center - Wakefield Campus,40.894207,-73.860790,Hospital


In [ ]:
not_hospitals = newyork_hospitals3[ (newyork_hospitals3['Venue Category'] != 'Hospital') & (newyork_hospitals3['Venue Category'] != 'Medical Center') & (newyork_hospitals3['Venue Category'] != 'Emergency Room')].index
newyork_hospitals3.drop(not_hospitals , inplace=True)
newyork_hospitals3.head()

In [ ]:
print(newyork_hospitals3.shape)

In [ ]:
newyork_numbers=newyork_hospitals.groupby('Neighborhood').count()
newyork_numbers.head()

In [ ]:
newyork_numbers.sort_values(by='Neighborhood Latitude', ascending=True)

In [ ]:
newyork_hospitals

In [ ]:
# create map of New York using latitude and longitude values
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
folium.Marker(
    location=[40.710668,-74.005411],
    popup='NY Presbyterian_Lower Manhattan',
    icon=folium.Icon(color='green')
    ).add_to(map_newyork)
folium.Marker(
    location=[40.707316,-74.005612],
    popup='NY Presbyterian Hospital',
    icon=folium.Icon(color='green')
    ).add_to(map_newyork)
folium.Marker(
    location=[40.716816,-74.007073],
    popup='Weill Cornell Medical Center',
    icon=folium.Icon(color='purple')
    ).add_to(map_newyork)
folium.Marker(
    location=[40.709985,-74.004720],
    popup='NY Presbyterian Emergency Room',
    icon=folium.Icon(color='red')
    ).add_to(map_newyork)
map_newyork

In [47]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
